In [1]:
# Import the standard modules
import datetime
import pathlib
import sqlite3

# Import installed modules
import pandas as pd
import numpy as np
import spiceypy

# Import the Python script func from the auxiliary folder
import sys
sys.path.insert(1, '../../auxiliary')
import data_fetch

In [2]:
# Set a local download path and the URL to the comet data from the Minor
# Planet Center
DL_PATH = 'raw_data/'
DL_URL = 'https://www.minorplanetcenter.net/Extended_Files/cometels.json.gz'

# Download the comet data and store them in the directory
data_fetch.download_file(DL_PATH, DL_URL)

In [3]:
# Load the SPICE kernel meta file
spiceypy.furnsh('kernel_meta.txt')

In [4]:
# Read the g-zipped json file with pandas read_json. The function allows one
# to read compressed data
c_df = pd.read_json('raw_data/cometels.json.gz', compression='gzip')

In [5]:
# First we parse the date and time information. The dataset contains two
# time related information: the date-time of the last perihelion passage and
# another variable called Epoch. However, "epoch" is not related to the mean
# anomaly related epoch and represents other time information in this case.
#
# For our "actual" Epoch case we need to create a UTC time string based on the
# date and time of the last perihelion passage (the time corresponds to a mean
# anomaly of 0 degrees). The Day is given in DAY.FRACTION_OF_DAY. We extract
# only the day
c_df.loc[:, 'EPOCH_UTC_DATE'] = \
    c_df.apply(lambda x: str(x['Year_of_perihelion']) + '-' \
                         + str(x['Month_of_perihelion']) + '-' \
                         + str(x['Day_of_perihelion']).split('.')[0], \
               axis=1)

# Now we need to parse the .FRACTION_OF_DAY given between (0.0, 1.0). First,
# create a place-holder date
pre_time = datetime.datetime(year=2000, month=1, day=1)

# Use the pre_time date-time object and add the days and fraction of days with
# the timedelta function from the datetime library. Extract only the time
# substring ...
c_df.loc[:, 'EPOCH_UTC_TIME'] = c_df['Day_of_perihelion'] \
                                    .apply(lambda x: (pre_time + datetime.timedelta(days=x)) \
                                                     .strftime('%H:%M:%S'))

# ... and based with the date, create now the UTC date-time
c_df.loc[:, 'EPOCH_UTC'] = c_df.apply(lambda x: x['EPOCH_UTC_DATE'] + 'T' + x['EPOCH_UTC_TIME'],\
                                      axis=1)

# Convert the UTC datetime to ET
c_df.loc[:, 'EPOCH_ET'] = c_df['EPOCH_UTC'].apply(lambda x: spiceypy.utc2et(x))

In [6]:
# Compute the semi-major axis for closed orbits ...
c_df.loc[:, 'SEMI_MAJOR_AXIS_AU'] = \
    c_df.apply(lambda x: x['Perihelion_dist'] / (1.0 - x['e']) if x['e'] < 1 else np.nan, axis=1)

# ... as well as the aphelion (if applicable)
c_df.loc[:, 'APHELION_AU'] = c_df.apply(lambda x: (1.0 + x['e']) * x['SEMI_MAJOR_AXIS_AU'] \
                                                  if x['e'] < 1 else np.nan, \
                                        axis=1)

In [7]:
# Create a sub-directory in the main directory of this repository, where a
# comet database shall be stored
pathlib.Path('../../databases/comets/').mkdir(parents=True, exist_ok=True)

# Create / Connect to a comet database and set the cursor
con = sqlite3.connect('../../databases/comets/mpc_comets.db')
cur = con.cursor()

# Create (if not existing) a comets' main table, where miscellaneous
# parameters are stored
cur.execute('CREATE TABLE IF NOT EXISTS ' \
            'comets_main(NAME TEXT PRIMARY KEY, ' \
                        'ORBIT_TYPE TEXT, ' \
                        'PERIHELION_AU REAL, ' \
                        'SEMI_MAJOR_AXIS_AU REAL, ' \
                        'APHELION_AU REAL, ' \
                        'ECCENTRICITY REAL, ' \
                        'INCLINATION_DEG REAL, ' \
                        'ARG_OF_PERIH_DEG REAL, ' \
                        'LONG_OF_ASC_NODE_DEG REAL, ' \
                        'MEAN_ANOMALY_DEG REAL DEFAULT 0.0, ' \
                        'EPOCH_UTC TEXT, ' \
                        'EPOCH_ET REAL, ' \
                        'ABSOLUTE_MAGNITUDE REAL, ' \
                        'SLOPE_PARAMETER REAL'
                        ')')

In [8]:
# Insert the data
cur.executemany('INSERT OR REPLACE INTO ' \
                'comets_main(NAME, ' \
                            'ORBIT_TYPE, ' \
                            'PERIHELION_AU, ' \
                            'SEMI_MAJOR_AXIS_AU, ' \
                            'APHELION_AU, ' \
                            'ECCENTRICITY, ' \
                            'INCLINATION_DEG, ' \
                            'ARG_OF_PERIH_DEG, ' \
                            'LONG_OF_ASC_NODE_DEG, ' \
                            'EPOCH_UTC, ' \
                            'EPOCH_ET, ' \
                            'ABSOLUTE_MAGNITUDE, ' \
                            'SLOPE_PARAMETER'
                            ') ' \
                'VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', \
                c_df[['Designation_and_name', \
                      'Orbit_type', \
                      'Perihelion_dist', \
                      'SEMI_MAJOR_AXIS_AU', \
                      'APHELION_AU', \
                      'e', \
                      'i', \
                      'Peri', \
                      'Node', \
                      'EPOCH_UTC', \
                      'EPOCH_ET', \
                      'H', \
                      'G']].values)

# Commit
con.commit()

# Close the database. The database shall be the fundament for the next
# tutorial sessions
con.close()